# Exploração e análise de dados de crédito com SQL

Os dados representam informações de clientes de um banco e contêm as seguintes colunas:

* idade = idade do cliente
* sexo = sexo do cliente (M ou F)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade dos clientes
* estado_civil = estado civil do cliente
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartão do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações feitas nos últimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de crédito do cliente
* valor_transacoes_12m = valor das transações nos últimos 12 meses
* qtd_transacoes_12m = quantidade de transações dos últimos 12 meses

A tabela de SQL foi criada usando o **AWS Athena** através de um arquivo colocado em um **Bucket S3**. 

## 1. Limpeza
Primeiramente, precisamos limpar o dataset colocando os valores decimais num formato legível pelo AWS Athena. Ou seja, vírgulas são substituídas por pontos. Além disso, de cerca de 10 mil linhas reduzimos a 2500 para menor custo de computação.

In [ ]:
import pandas as pd

df = pd.read_csv('../input/credit-data/credito.csv', sep=',')

# Limpeza de dados
cols = ['limite_credito', 'valor_transacoes_12m']

for col in cols:
  df[col] = df[col].apply(lambda x: x.replace('.', '').replace(',', '.'))

# Removendo colunas que não são úteis na análise
df = df.drop(['id', 'default', 'meses_de_relacionamento'], axis=1)

# Limitando a quantidade de dados a 2500 linhas e sem cabeçalho
df = df.iloc[0:2500]
df.to_csv('../working/credito_clean.csv', sep=',', header=False, index=False)

## 2. Criação da tabela e descrição

No AWS S3 é criado um novo bucket s3://bucket-projeto-sql/ onde é colocado o arquivo credito_clean.csv. Na AWS Athena é criada a tabela 'credito' a partir deste arquivo csv.

```
CREATE EXTERNAL TABLE credito(
  idade INT,
  sexo STRING,
  dependentes INT,
  escolaridade STRING,
  estado_civil STRING,
  salario_anual STRING,
  tipo_cartao STRING,
  qtd_produtos BIGINT,
  iteracoes_12m INT,
  meses_inativo_12m INT,
  limite_credito FLOAT,
  valor_transacoes_12m FLOAT,
  qtd_transacoes_12m INT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('serialization.format' = ',', 'field.delim' = ',')
LOCATION 's3://bucket-projeto-sql/'
TBLPROPERTIES('has_encrypted_data' = 'false');
```

Através da query SQL

```
DESCRIBE credito;
```

é possível ver todas as colunas da tabela formada e seus tipos de dados.

<img src='https://drive.google.com/uc?id=1Rg_WMnTuh6OWKFjqFg0RZavbkUgcvoVB' style='display:block;float:none;margin-left:auto;margin-right:auto;width:30%'/>

## 3. Exploração de dados

### 3.1 - Categorias de dados

A query

```
SELECT DISTINCT escolaridade
FROM credito;
```

mostra que existem **cinco tipos de escolaridade** na tabela: sem educação formal, ensino medio, graduação, mestrado e doutorado. O tipo 'na' apenas indica erro na hora de processar a informação num cadastro, ou dados faltantes. Logo, precisa ser filtrada em outras consultas.

<img src='https://drive.google.com/uc?id=1nUHVfGsQ3YRFKqmiUVX7ThPPGTI_oqhm' style='display:block;float:none;margin-left:auto;margin-right:auto;width:30%'/>

Com outra query

```
SELECT DISTINCT estado_civil
FROM credito;
```

<img src='https://drive.google.com/uc?id=14z1safnNen4nJSC8p0bKykC9r7WLy8dU' style='display:block;float:none;margin-left:auto;margin-right:auto;width:25%'/>

Consegue-se ver que existem **três tipos de estados civis** disponíveis: solteiro, casado ou divorciado e 'na' dos dados faltantes.

Já pela consulta

```
SELECT DISTINCT salario_anual
FROM credito;
```

<img src='https://drive.google.com/uc?id=1ftnzhW4WHFVS-fit64WXjSDWIzbwvz8b' style='display:block;float:none;margin-left:auto;margin-right:auto;width:25%'/>

Para salário anual temos cerca de **cinco faixas salariais**: menos que \\$40K, \\$40K - \\$60K, \\$60K - \\$80K, \\$80K - \\$120K, \\$120K + e 'na'.

### 3.2 - Agregações

Uma forma de organizar as informações é agrupar os dados através destas categorias. Com a query

```
SELECT COUNT(*) AS qtde, salario_anual
FROM credito
GROUP BY salario_anual
ORDER BY qtde ASC;
```

Conseguimos ver que o maior grupo de pessoas está na faixa salarial de 'menos que \\$40K' e que uma minoria possui salário na faixa de '\\$120K +'.

<img src='https://drive.google.com/uc?id=1yXR6MxMDdvz4WU06BBRbV1E2i1R8IDcR' style='display:block;float:none;margin-left:auto;margin-right:auto;width:50%'/>

O gráfico de pizza a seguir ilustra bem esses grupos salariais.

<img src='https://drive.google.com/uc?id=1Suge5IIqgITv-yBb7Zhc536hJaExXuN5' style='display:block;float:none;margin-left:auto;margin-right:auto;width:60%'/>

Outro tipo de agregação útil é contar a quantidade de clientes por sexo utilizando determinado tipo de cartão. A query para isto é

```
SELECT COUNT(*) AS qtde, sexo, tipo_cartao
FROM credito
GROUP BY sexo, tipo_cartao
ORDER BY qtde DESC;
```

<img src='https://drive.google.com/uc?id=1xWeGHpSXAWsYUnro3LVbScpjSlG1hEYc' style='display:block;float:none;margin-left:auto;margin-right:auto;width:60%'/>

A partir desta consulta vemos que a maioria dos clientes usa o cartão 'blue', e que os homens são a maior parte dos usuários de todos os tipos de cartão. Porém, isto também pode se dever ao conjunto de dados conter mais homens do que mulheres. O gráfico de barra a seguir deixa esta diferença mais evidente.

<img src='https://drive.google.com/uc?id=1NYYDDZ69iUqmdg22U00tpkhBry92AaYS' style='display:block;float:none;margin-left:auto;margin-right:auto;width:70%'/>